In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

In [2]:
#Read in the data
df = pd.read_csv(r"C:\Users\Peter Villarreal\Desktop\Data Analytics\Datasets\texas_car_crashes\texas_car_crashes - texas_car_crashes.csv")
#Lets look at the data
df.head()

C:\Users\Peter Villarreal\AppData\Local\Temp\ipykernel_19996\458745912.py:2: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Peter Villarreal\Desktop\Data Analytics\Datasets\texas_car_crashes\texas_car_crashes - texas_car_crashes.csv")


,Crash ID,$1000 Damage to Any One Person's Property,Active School Zone Flag,Adjusted Average Daily Traffic Amount,Adjusted Roadway Part,At Intersection Flag,Average Daily Traffic Amount,Average Daily Traffic Year,Bridge Detail,City,...,Vehicle Damage Rating 1 - Severity,Vehicle Defect 1,Citation,Person Age,Person Alcohol Result,Person Drug Test Result,Person Ethnicity,Person Gender,Person Injury Severity,Person Type
0,18038521,Yes,NO,127485,1 - MAIN/PROPER LANE,False,127485,2019,NOT APPLICABLE,AUSTIN,...,1 - DAMAGED 1 MINIMUM,No Data,No Data,22,No Data,97 - NOT APPLICABLE,B - BLACK,2 - FEMALE,N - NOT INJURED,1 - DRIVER
1,18042943,Yes,NO,126150,1 - MAIN/PROPER LANE,False,126150,2019,NOT APPLICABLE,AUSTIN,...,4 - DAMAGED 4,No Data,No Data,22,No Data,97 - NOT APPLICABLE,W - WHITE,1 - MALE,N - NOT INJURED,1 - DRIVER
2,18042943,Yes,NO,126150,1 - MAIN/PROPER LANE,False,126150,2019,NOT APPLICABLE,AUSTIN,...,4 - DAMAGED 4,No Data,No Data,35,No Data,97 - NOT APPLICABLE,H - HISPANIC,1 - MALE,N - NOT INJURED,1 - DRIVER
3,18042945,No,NO,No Data,1 - MAIN/PROPER LANE,False,No Data,No Data,NOT APPLICABLE,AUSTIN,...,2 - DAMAGED 2,No Data,No Data,120,No Data,97 - NOT APPLICABLE,99 - UNKNOWN,99 - UNKNOWN,99 - UNKNOWN,99 - UNKNOWN
4,18042945,No,NO,No Data,1 - MAIN/PROPER LANE,False,No Data,No Data,NOT APPLICABLE,AUSTIN,...,1 - DAMAGED 1 MINIMUM,No Data,No Data,120,No Data,97 - NOT APPLICABLE,99 - UNKNOWN,99 - UNKNOWN,99 - UNKNOWN,99 - UNKNOWN


In [3]:
vars_coordinates = df[["Latitude", "Longitude", "Person Age", "Person Gender", "Person Alcohol Result"]]
vars_coordinates.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,No Data
1,30.35356763,-97.7146111,22,1 - MALE,No Data
2,30.35356763,-97.7146111,35,1 - MALE,No Data
3,30.22705309,-97.72169457,120,99 - UNKNOWN,No Data
4,30.22705309,-97.72169457,120,99 - UNKNOWN,No Data


In [5]:
semi_NaNs = pd.DataFrame(vars_coordinates, columns = ["Latitude", "Longitude", "Person Age", "Person Gender", "Person Alcohol Result"])
semi_NaNs = vars_coordinates.replace(["NaN", "99 - UNKNOWN", "No Data"], "NaN", regex=True)
semi_NaNs.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,NaN
1,30.35356763,-97.7146111,22,1 - MALE,NaN
2,30.35356763,-97.7146111,35,1 - MALE,NaN
3,30.22705309,-97.72169457,120,NaN,NaN
4,30.22705309,-97.72169457,120,NaN,NaN


In [7]:
semi_NaNs.groupby("Longitude").count()

,Latitude,Person Age,Person Gender,Person Alcohol Result
Longitude,,,,
-97.58013409,1,1,1,1
-97.5802294,3,3,3,3
-97.58031464,5,5,5,5
-97.58035563,8,8,8,8
-97.5807543,2,2,2,2
...,...,...,...,...
-97.91323378,2,2,2,2
-97.9132416,2,2,2,2
-97.91921242,1,1,1,1


In [8]:
age = semi_NaNs["Person Age"].str.len()
semi_NaNs.loc[age > 2, "Person Age"] = np.nan
semi_NaNs.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,NaN
1,30.35356763,-97.7146111,22,1 - MALE,NaN
2,30.35356763,-97.7146111,35,1 - MALE,NaN
3,30.22705309,-97.72169457,NaN,NaN,NaN
4,30.22705309,-97.72169457,NaN,NaN,NaN


In [9]:
df_semiclean = semi_NaNs.dropna()
df_semiclean.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,NaN
1,30.35356763,-97.7146111,22,1 - MALE,NaN
2,30.35356763,-97.7146111,35,1 - MALE,NaN
5,30.29983476,-97.73209245,21,2 - FEMALE,1 - POSITIVE
6,30.33227988,-97.68661638,24,1 - MALE,NaN


In [10]:
df_semiclean.groupby("Longitude").count()

,Latitude,Person Age,Person Gender,Person Alcohol Result
Longitude,,,,
-97.58013409,1,1,1,1
-97.5802294,3,3,3,3
-97.58031464,5,5,5,5
-97.58035563,7,7,7,7
-97.5807543,2,2,2,2
...,...,...,...,...
-97.90977721,1,1,1,1
-97.91323378,2,2,2,2
-97.9132416,2,2,2,2


In [14]:
values_list = ['NaN']

noNaN_long = df_semiclean[~df_semiclean['Longitude'].isin(values_list)]
noNaN_long.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,NaN
1,30.35356763,-97.7146111,22,1 - MALE,NaN
2,30.35356763,-97.7146111,35,1 - MALE,NaN
5,30.29983476,-97.73209245,21,2 - FEMALE,1 - POSITIVE
6,30.33227988,-97.68661638,24,1 - MALE,NaN


In [16]:
values_list = ['NaN']

noNaN_lat = noNaN_long[~noNaN_long['Latitude'].isin(values_list)]
noNaN_lat.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
0,30.44442854,-97.66771599,22,2 - FEMALE,NaN
1,30.35356763,-97.7146111,22,1 - MALE,NaN
2,30.35356763,-97.7146111,35,1 - MALE,NaN
5,30.29983476,-97.73209245,21,2 - FEMALE,1 - POSITIVE
6,30.33227988,-97.68661638,24,1 - MALE,NaN


In [17]:
noNaN_lat.groupby("Latitude").count()

,Longitude,Person Age,Person Gender,Person Alcohol Result
Latitude,,,,
30.10022079,4,4,4,4
30.10090545,6,6,6,6
30.10420176,2,2,2,2
30.10532182,3,3,3,3
30.1062179,2,2,2,2
...,...,...,...,...
30.50253848,1,1,1,1
30.50559611,1,1,1,1
30.50561779,2,2,2,2


In [18]:
alcohol_results = noNaN_lat["Person Alcohol Result"].isin(["1 - POSITIVE", "2 - NEGATIVE"])
no_NaN_df = noNaN_lat[alcohol_results]
no_NaN_df.head()

,Latitude,Longitude,Person Age,Person Gender,Person Alcohol Result
5,30.29983476,-97.73209245,21,2 - FEMALE,1 - POSITIVE
74,30.21642876,-97.75132751,29,1 - MALE,1 - POSITIVE
136,30.35716476,-97.72395246,29,2 - FEMALE,1 - POSITIVE
435,30.21621963,-97.7442951,41,1 - MALE,1 - POSITIVE
667,30.29486322,-97.7196339,24,1 - MALE,1 - POSITIVE
